In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial import Voronoi, voronoi_plot_2d, KDTree
from scipy.ndimage import map_coordinates

from skimage.io import imread, imsave
from skimage import feature, filters, measure
from skimage.color import rgb2gray
from skimage.filters.rank import gradient
from skimage.transform import rescale, rotate
from skimage.util import crop
from matplotlib import pyplot as plt
from matplotlib.pyplot import Axes

In [ ]:
#image = imread("./data/input/42501f8a-5015-41e2-a87b-2980df8237a4.webp")/255.0
#image = imread("./data/input/a4ca9877-c77b-4a8e-9d7a-aa621a7736fe.webp")/255.0
#image = imread("./data/input/pexels-photo-2382325.jpeg")/255.0
image = imread("./data/input/1c7cy8.jpg")/255.0
# image = imread("./data/input/carpet.webp")
# image = imread("./data/input/abstract-pattern-in-dark-pastel-tones-purple-background-and-green-blue-and-pink-spirals-vector.jpg")
#image = imread("./data/input/Cge6pcFRQLmteLQyGiIznQ.webp")
#image = rotate(image,90, resize=True)

image = crop(image,[(100,400),(500,300),(0,0)])
plt.imshow(image)

In [ ]:
# image_width, image_height, _ = image.shape
NUMBER_OF_POINTS = 8000

In [ ]:
def draw_simulated_points(ax: Axes, heightmap: np.ndarray, points_xy: np.ndarray, show_voronoi=False):
    ax.imshow(heightmap)
    ax.scatter(x=points_xy[:, 1], y=points_xy[:, 0], s=0.2, c="red")
    if show_voronoi:
        voronoi_plot_2d(
            Voronoi(points_xy[:,::-1]),
            ax=ax,
            show_vertices=False,
            show_points=False,
            line_colors='red',
            line_alpha=0.5,
            line_width=0.8,
            point_size=0.2,
        )
    ax.set_xlim((0, heightmap.shape[1]))  # width
    ax.set_ylim((0, heightmap.shape[0]))  # height
    ax.axis("off")
    ax.invert_yaxis()
    return ax

In [ ]:
def draw_plain_image(ax:Axes, image:np.ndarray):
    ax.imshow(image)
    ax.axis("off")
    ax.set_xlim([0, image.shape[1]])
    ax.set_ylim([0, image.shape[0]])
    ax.invert_yaxis()

In [ ]:
def adjust_points_via_forcefield_kdtree(points, force_field_x, force_field_y, iterations=30, force_multiplier=50, repulsion_multiplier=10, kd_tree_update_freq=5, kd_tree_query_radius=50):
    """
    Adjusts points based on a simulated force field and KD Tree for efficient nearest neighbor search.
    
    Parameters:
    - points: numpy.ndarray, the original points as an Nx2 array.
    - force_field_x: numpy.ndarray, the x component of the force field.
    - force_field_y: numpy.ndarray, the y component of the force field.
    - iterations: int, number of iterations to apply the force field adjustments.
    - force_multiplier: float, multiplier for the force field effect.
    - repulsion_multiplier: float, multiplier for the repulsion effect.
    - kd_tree_update_freq: int, frequency of KD tree rebuilds per iteration.
    
    Returns:
    - numpy.ndarray, the adjusted points.
    """
    voronoi_point_xy = points.copy()
    kd_tree = KDTree(voronoi_point_xy)  # Initial KD tree build

    for iteration in range(iterations):
        # Rebuild KD tree periodically to accommodate for moved points
        if iteration % kd_tree_update_freq == 0:
            kd_tree = KDTree(voronoi_point_xy)

        force_x = map_coordinates(force_field_x, voronoi_point_xy.transpose())
        force_y = map_coordinates(force_field_y, voronoi_point_xy.transpose())
        
        # Use KD tree to find points within a certain radius
        # This radius should be chosen based on your dataset and the expected repulsion range
        
        neighbors_list = kd_tree.query_ball_point(voronoi_point_xy, r=kd_tree_query_radius)

        repulsion_vectors = np.zeros_like(voronoi_point_xy)
        for i, neighbors in enumerate(neighbors_list):
            if neighbors:
                diff = voronoi_point_xy[i] - voronoi_point_xy[neighbors]
                dist_squared = np.sum(diff ** 2, axis=1)
                epsilon = 1e-5
                repulsion_strength = 1 / (dist_squared + epsilon)
                repulsion_vectors[i] = np.sum(diff * repulsion_strength[:, None] / np.sqrt(dist_squared + epsilon)[:, None], axis=0)

        # Apply the forces
        voronoi_point_xy += np.stack([force_x, force_y], axis=-1) * force_multiplier + repulsion_vectors * repulsion_multiplier

    return voronoi_point_xy

In [ ]:
def voronoi_pixelate(image, points, upscale_factor=1):
    """
    Computes the average color of Voronoi cells in an image based on a set of points and upscales the result.
    
    Parameters:
    - image: numpy.ndarray, the input image as an array of shape (height, width, 3).
    - points: numpy.ndarray, the original points defining Voronoi cells as an Nx2 array.
    - upscale_factor: int, the factor by which to upscale the image (default is 1, which means no scaling).
    
    Returns:
    - numpy.ndarray, the upscaled image where each pixel's color is the average color of its Voronoi cell.
    """
    # Upscale the image if needed
    if upscale_factor != 1:
        image_upscaled = rescale(image, (upscale_factor, upscale_factor, 1), anti_aliasing=True)
    else:
        image_upscaled = image

    # Build a KDTree from the points
    tree = KDTree(points)

    # Generate grid for upscaled image and adjust for the upscale
    x_upscaled, y_upscaled = np.meshgrid(range(image.shape[1] * upscale_factor), range(image.shape[0] * upscale_factor))
    yx_indexes_flat_upscaled = np.stack([y_upscaled.flatten(), x_upscaled.flatten()], axis=-1) 

    # Query the KDTree, adjusting for the upscale
    _, upscaled_group_index = tree.query(yx_indexes_flat_upscaled / upscale_factor)

    # Find unique groups and their indexes in the upscaled image
    upscaled_unique_groups, upscaled_unique_groups_indexes = np.unique(upscaled_group_index, return_inverse=True)

    # Initialize arrays for total pixel colors and counts in the upscaled image
    total_pixel_colors = np.zeros((*upscaled_unique_groups.shape, 3))
    total_pixel_counts = np.zeros(upscaled_unique_groups.shape)

    # Sum colors and counts for each group
    np.add.at(total_pixel_colors, upscaled_unique_groups_indexes, image_upscaled.reshape(-1, 3))
    np.add.at(total_pixel_counts, upscaled_unique_groups_indexes, 1)

    # Compute the average color for each group
    average_color = total_pixel_colors / total_pixel_counts.reshape(-1, 1)

    # Assign the average color back to the pixels based on their group in the upscaled image
    upscaled_pixel_values = average_color[upscaled_unique_groups_indexes].reshape((image.shape[0] * upscale_factor, image.shape[1] * upscale_factor, 3))

    return upscaled_pixel_values

In [ ]:


# Ensure the input image is grayscale for the force field computation
simulated_heightmap = filters.difference_of_gaussians(rgb2gray(image), low_sigma=6)

# Compute the gradient (force field) of the height map
fx = filters.sobel_h(simulated_heightmap)
fy = filters.sobel_v(simulated_heightmap)

plt.imshow(simulated_heightmap)

In [ ]:
points = np.random.normal(loc=[0.4,0.3], scale=[0.4,0.4], size=[NUMBER_OF_POINTS*5,2])
points = points[np.bitwise_and.reduce((points>0) & (points<1),axis=-1)]
points = points[np.random.choice(points.shape[0], size=NUMBER_OF_POINTS, replace=False)]
ax = plt.scatter(x=points[:,1],y=1-points[:,0])
points.shape

In [ ]:

#voronoi_point_xy_original = np.random.rand(NUMBER_OF_POINTS, 2) * np.array(image.shape[:2])
voronoi_point_xy_original = points * np.array(image.shape[:2])

voronoi_point_xy_light_seekers, voronoi_point_xy_dark_seekers = np.array_split(voronoi_point_xy_original,2)


# run main simulation to redistribute points:
voronoi_point_xy_light_seekers = adjust_points_via_forcefield_kdtree(voronoi_point_xy_light_seekers,iterations=100, force_field_x=fx, force_field_y=fy, force_multiplier= 100, repulsion_multiplier=10)
voronoi_point_xy_dark_seekers  = adjust_points_via_forcefield_kdtree(voronoi_point_xy_dark_seekers ,iterations=100, force_field_x=fx, force_field_y=fy, force_multiplier=-100, repulsion_multiplier=10)


voronoi_point_xy = np.concatenate([
    voronoi_point_xy_light_seekers,
    voronoi_point_xy_dark_seekers
])
adjust_points_via_forcefield_kdtree(voronoi_point_xy,iterations=3, force_field_x=fx, force_field_y=fy, force_multiplier=2, repulsion_multiplier=50)

In [ ]:
ordinary_new_image = voronoi_pixelate(image, voronoi_point_xy_original)
special_new_image = voronoi_pixelate(image, voronoi_point_xy)

In [ ]:
fig, axs = plt.subplots(2,2)
axs = axs.flatten()

# original Image, with original points
axs[0].set_title("Original Image with Random Points")
draw_simulated_points(axs[0], image, voronoi_point_xy_original,show_voronoi=True)
#draw_simulated_points(axs[1], simulated_heightmap, voronoi_point_xy, show_voronoi=True)
axs[1].set_title("Poor Voronoi Pixelation  effect")
draw_plain_image(axs[1],ordinary_new_image)


#draw_simulated_points(axs[0], image, voronoi_point_xy_original)
axs[2].set_title("Sobel with point movement simulation")
draw_simulated_points(axs[2], simulated_heightmap, voronoi_point_xy, show_voronoi=True)
draw_plain_image(axs[3],special_new_image)
axs[3].set_title("Improved Voronoi Pixelation Effect")

fig.set_figheight(13)
fig.set_figwidth(20)
plt.tight_layout()

In [ ]:
upscaled = voronoi_pixelate(image,voronoi_point_xy_original,upscale_factor=4)
imsave("./data/output/ficus_leaves_other.jpg",(upscaled*255).astype("u1"))
plt.imshow(upscaled)

In [ ]:
upscaled = voronoi_pixelate(image,voronoi_point_xy,upscale_factor=4)
imsave("./data/output/ficus_leaves.png",(upscaled*255).astype("u1"))
plt.imshow(upscaled)